In [6]:
from cellcalc import get_primitive_hkl, MID, get_pri_vec_inplane
from interface_generator import core, print_near_axis, convert_vector_index,d_hkl,three_dot,cross_plane, get_right_hand
from csl_generator import print_list,get_theta_m_n_list, degrees
from numpy import array
from numpy import array, dot, round
from numpy.linalg import inv, norm
import numpy as np
import glob
import shutil
import os
from cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, MID
from interface_generator import core, print_near_axis, convert_vector_index, write_trans_file, get_ang_list
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv

import numpy as np
import pandas as pd
import shutil
import os
import math
import glob
from scipy.spatial import distance
import matplotlib.pyplot as plt
from numpy.linalg import inv, norm
from numpy import dot
from abc import ABCMeta, abstractclassmethod
import copy
import itertools
import collections

In [7]:
class Element2:
    def __init__(self,name,dirname,crystal_structure):
        self.name = name
        self.dirname = dirname
        self.grandenergy = self.getLatE(dirname)
        self.LP = self.getLatP(dirname)
        self.mass = self.get_mass(dirname)
        self.ciffile = glob.glob(f"{dirname}/*.cif")[0]
        self.crystal_structure = crystal_structure # this is a class representing crystal structure
#         with open(f"{self.dirname}/proto.in","r") as f:
#             self.proto_text = f.read() 
    
    def potential(self):
        for dirs in glob.glob(f"{self.dirname}/potential/*"):
            shutil.copy(f"{dirs}",f"{os.getcwd()}")
        
#     def proto(self):
#         shutil.copy(f"{self.dirname}/proto.in",f"{os.getcwd()}")
    
    @staticmethod
    def getLatE(dirname):
        """
        read the Energy of single crystal 
        """
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatE = lines[-4:][0].split()[4].replace(';', '')
        return float(LatE)

    @staticmethod
    def getLatP(dirname):
        """
        read the Lattice Parameter
        """
        with open(f'{dirname}/atomsout','r') as f:
            lines=f.readlines()
            LatP = lines[-5:][0].split()[4].replace(';', '')
        return float(LatP)

    @staticmethod
    def get_mass(dirname):
        with open(f'{dirname}/mass','r') as f:
            lines=f.read()
        return float(lines)

In [8]:
class ProtoinCreator(metaclass=ABCMeta):
    # element →　Element クラス　オブジェクト　インスタンス化"前"
    # SanplingMethod　→　SamplingMethodCreator クラス　オブジェクト　インスタンス化"前"
    def __init__(self,element,SamplingMethod):
        self.element = element
        self.sampling_method = SamplingMethod
        self.vanishing_rule = element.crystal_structure.vanishing_rule # this is function
#         self.crystal_structurte = self.element.crystal_structure
        # で、僕はこのself.crystal_structureが持っているメソッドを使いたい
    def generate(self,axis,hkl):
        samplingmethod = self.sampling_method(axis,hkl) # インスタンス化後のSamplingMethodCreatorクラスメソッド
        samplingmethod.vanishing_rule = self.vanishing_rule # passしていたvanishing_ruleをオーバーライド
        protoinbodycreator = ProtoinBodyCreator(samplingmethod)
        self.body_text = protoinbodycreator.generate(self.element.name)
    # サブクラスがインスタンス化されるまで保留して一旦設計を進めることができる
    
    @abstractclassmethod
    def header(self):
        pass
    
    @abstractclassmethod
    def body(self):
        pass 
    
    
    #テンプレートの役割を持つクラス
    def create(self):
        h = self.header()
        b = self.body()
        return f"{h}\n\n{b}"

In [9]:
class Protoin(ProtoinCreator):
    
    # 継承先を固定しておけばテンプレートを動かさなくていい
    def header(self):
        text = f"""


variable element_mass equal {self.element.mass} 

variable minimumenergy equal {self.element.grandenergy}

variable lattice_param equal {self.element.LP}
                """
        
        return text

    
    def body(self):
        return self.body_text
    

In [10]:
class ProtoinBodyCreator:
    def __init__(self,sampling_method_creator):
        self.sampling = sampling_method_creator
        
    def generate(self,element):
        return self.sampling.calc_condition(element)

In [32]:
class Calc_Condition:
    def __init__(self,element,SamplingMethod,cell_size,fix_bulk_region,middle_region,start_bulk,end_bulk):

#         self.cnid_sampling = cnid_sampling
        self.element = element
#        self.protoin = Protoin(self.element,SamplingMethod)
        self.protoin = Protoin(self.element,SamplingMethod)
        self.cell_size = cell_size
        self.fix_bulk_region = fix_bulk_region
        self.middle_region = middle_region
        self.start_bulk = start_bulk
        self.end_bulk = end_bulk
        
    def define_bicrystal_regions(self,xhi):
        tol = 1e-5
        """
        generate a file defining some regions in the LAMMPS and define the atoms
        inside these regions into some groups.
        argument:
        region_names --- list of name of regions
        region_los --- list of the low bounds
        region_his --- list of the hi bounds
        """
        end_fixbulk1 = xhi/2-self.fix_bulk_region
        start_fixbulk2 = xhi/2+self.fix_bulk_region
        start_middle = xhi/2-self.middle_region
        end_middle = xhi/2+self.middle_region
        start_right = xhi/2 - tol
        start_bulk = self.start_bulk
        end_bulk = self.end_bulk


        with open('blockfile', 'w') as fb:
            fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(end_fixbulk1))
            fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_fixbulk2))
            fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_middle,end_middle))
            fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region left block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_right))
            fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
            format(start_bulk,end_bulk))
            fb.write('group fixbulk1 region fixbulk1 \n')
            fb.write('group fixbulk2 region fixbulk2 \n')
            fb.write('group middle region middle \n')
            fb.write('group right region right \n')
            fb.write("group left region left \n")
            fb.write('group bulk region bulk \n')
            fb.write("variable startright equal {0:.16f}".\
            format(start_right))
        
    def define_bicrystal_regions2(self,xhi):
        half_cell = xhi/2
        text = f"""
    displace_atoms left move ${{tx1}} ${{ty1}} ${{tz1}} units box

    group right_dash region right 

    group deleted_atoms subtract right_dash right

    displace_atoms deleted_atoms move {half_cell} 0 0 units box

    displace_atoms right move ${{tx2}} ${{ty2}} ${{tz2}} units box

    group left_dash region left 

    group deleted_atoms subtract left_dash left

    displace_atoms deleted_atoms move -{half_cell} 0 0 units box

    group fixbulk1_dash region fixbulk1 
    group fixbulk2_dash region fixbulk2 
    group middle_dash region middle 
    group bulk_dash region bulk

                """
        with open("blockfile2", "w") as f:
            f.writelines(text)
        
    def generate_files(self,axis,hkl,xhi):
        self.protoin.generate(axis,hkl)
        self.element.potential()
        self.define_bicrystal_regions(xhi)
        self.define_bicrystal_regions2(xhi)
        with open("proto.in","w") as f:
            f.writelines(self.protoin.create())
        
#         if self.cnid_sampling==True:
#             self.element.proto()
#         else:
#             with open("proto.in","w") as f:
#                 f.writelines(self.protoin.create())

In [12]:
class core2(core):
    def __init__(self, file_1, file_2):
        super().__init__(file_1,file_2)
    def compute_stgb(self, axis, plane_n, lim = 20, tol = 1e-10):
        """
        compute the transformation to obtain the supercell of the two slabs forming a interface
        argument:
        hkl --- miller indices of the plane expressed in lattice_1
        lim --- the limit searching for a CSL vector cross the plane
        normal_ortho --- whether limit the vector crossing the GB to be normal to the GB
        plane_ortho --- whether limit the two vectors in the GB plane to be orthogonal
        tol --- tolerance judging whether orthogonal
        """
        hkl = MID(self.lattice_1,plane_n)
        self.d1 = d_hkl(self.lattice_1, hkl)
        lattice_2 = three_dot(self.R, self.D, self.lattice_2)
        hkl_2 = get_primitive_hkl(hkl, self.lattice_1, lattice_2)
        self.d2 = d_hkl(lattice_2, hkl_2)
        hkl_c = get_primitive_hkl(hkl, self.lattice_1, self.CSL) # miller indices of the plane in CSL
        hkl_c = np.array(hkl_c)
        
        #plane_n = dot(self.lattice_1,hkl)
        axis_cartesian = dot(self.conv_lattice_1,axis)
        v1 = cross_plane(lattice=self.CSL, n=plane_n, lim=lim,  orthogonal=True, tol=tol) # a CSL basic vector cross the plane
        v2 = cross_plane(lattice=self.CSL, n=axis_cartesian, lim=lim, orthogonal=True, tol=tol)
        v3_cartesian = cross(v1,v2)
        v3 = cross_plane(lattice=self.CSL, n=v3_cartesian, lim=lim, orthogonal=True, tol=tol)
        supercell = np.column_stack((v1,v2,v3)) # supercell of the bicrystal
        supercell = get_right_hand(supercell) # check right-handed


        self.bicrystal_U1 = np.array(np.round(dot(inv(self.lattice_1), supercell),8),dtype = int)
        self.bicrystal_U2 = np.array(np.round(dot(inv(self.lattice_2_TD), supercell),8),dtype = int)
        self.cell_calc.compute_CNID(hkl)
        CNID = self.cell_calc.CNID
        self.CNID = dot(self.lattice_1, CNID)

In [13]:
  class MyInterface:
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        
        self.my_interface = core2(self.cif_filename, self.cif_filename)
        
        self.my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=self.max_sigma, sgm2=self.max_sigma, dd = 1e-4)
        factor = self.LP/(norm(self.my_interface.conv_lattice_1[:,0]))
        self.my_interface.lattice_1 =  self.my_interface.lattice_1*factor
        self.my_interface.lattice_2 =  self.my_interface.lattice_2*factor
        self.my_interface.conv_lattice_1 =  self.my_interface.conv_lattice_1*factor
        self.my_interface.conv_lattice_2 =  self.my_interface.conv_lattice_2*factor
        

In [14]:
class Factory(object): # ここはインスタンスを生成する工場だから普段インスタンスを作ることをやる
    def create(self,hkl,delivery_point="."): # インスタンスをcreate
        # 回転軸、回転角、plane_nを受け取って作成
        axis, theta, plane_n = self.get_gb_info_from_hkl(hkl)
        self.createGB(axis,theta,plane_n,hkl,delivery_point)
    
    def createGB(self,axis,theta,plane_n,hkl,delivery_point):
        # interfaceインスタンス化 + get_gbfiles実行
        pass

    
    def generate_hkl_list(self,axis,max_sigma):
        pass
    
    def get_gb_info_from_hkl(self,hkl):
        pass
    


In [15]:
class GBFactory(Factory):
    def __init__(self,max_sigma,axis,calc_condition):
        self.calc_condition = calc_condition
        self.cif_filename = self.calc_condition.element.ciffile
        self.LP = self.calc_condition.element.LP
        self.max_sigma = max_sigma
        self.axis = axis
        self.primitive_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        self.conventional_interface = MyInterface(max_sigma,axis,calc_condition).my_interface
        
    def createGB(self,axis,theta,plane_n,hkl,delivery_point):
    # interface serch csl create_gbfiles実行
        axis_cart = dot(self.primitive_interface.conv_lattice_1, np.array(axis))

        axis_pri = np.array(np.round(dot(inv(self.primitive_interface.lattice_1), axis_cart),8),dtype = int)
        print(axis_pri)
        print(theta)
        print(self.primitive_interface.lattice_1)
        
        self.primitive_interface.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)
        self.get_gb_files(axis,plane_n,hkl,delivery_point)
        
#         get_gb_files(self,interface,axis,hkl,calc_condition,input_dirname=None)
    
    @classmethod
    def generate_hkl_list(cls,axis,max_sigma,calc_condition):
        conventional_interface = copy.deepcopy(cls(max_sigma,axis,calc_condition).conventional_interface)
        conventional_interface.lattice_1 = conventional_interface.conv_lattice_1
        conventional_interface.lattice_2 = conventional_interface.conv_lattice_2
        
        thetas = cls.get_thetas(axis,max_sigma)

        gb_dict = {}

        for theta in thetas:

            conventional_interface.search_one_position(np.array(axis),theta-0.0000001,1,0.0000001)
            # n_1, n_2をリストにする
            n_list = cls.get_a_b(conventional_interface.CSL, dot(conventional_interface.lattice_1,np.array(axis)))
            
            for n_1 in n_list:
                # リストからfor ループでn_1を取り出す
                hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_1),8),dtype = int)
                hkl_name = np.array(hkl_name, dtype=int)
                hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
                hkl_x, hkl_y, hkl_z = hkl_name
                axis_x, axis_y, axis_z = axis

                gb_info1 = {f"{hkl_x}_{hkl_y}_{hkl_z}":[axis,theta,n_1]}
                gb_dict = {**gb_dict,**gb_info1}
                print(hkl_name)

#             hkl_name = np.array(np.round(dot(inv(conventional_interface.conv_lattice_1), n_2),8),dtype = int)
#             hkl_name = np.array(hkl_name, dtype=int)
#             hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
#             hkl_x, hkl_y, hkl_z = hkl_name
#             axis_x, axis_y, axis_z = axis

#             gb_info2 = {f"{hkl_x}_{hkl_y}_{hkl_z}":[axis,theta,n_2]}

#             if axis==[1,1,1,]:
#                 gb_dict = {**gb_dict,**gb_info1}
#             else:
#                 gb_dict = {**gb_dict,**gb_info1,**gb_info2}
            
            print(list(gb_dict.keys()))
        
        cls.gb_dict = gb_dict
        
        return cls(max_sigma,axis,calc_condition)
    
    def get_gb_info_from_hkl(self,hkl):
        return self.gb_dict[hkl]
     
    @staticmethod
    def get_thetas(axis,max_sigma):
        theta_list = []
        sigma_list = []
        lists, thetas = GBFactory.getsigmas(axis, max_sigma)
        return thetas
    
    @staticmethod
    def get_csl_basis(CSL,axis):
        tol = 1e-8
        dot_list = get_ang_list(CSL.T,axis)
        normal_v = CSL.T[np.where(abs(dot_list - 1) > tol)[0]]
        return normal_v
    
    @staticmethod
    def get_a_b(CSL, axis):
#         hkl_perp_axis = MID(CSL, axis)
        n_list = GBFactory.v_expand(CSL)
        
#         a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T
        
#         if (norm(cross(axis,[1,1,1])) < 1e-8):
#             v1, v2 = GBFactory.get_csl_basis(CSL,axis)
#             if dot(v1,v2) < 0:
#                 v2 = -v2
        
#         if(norm(cross(axis,[1,0,0])) < 1e-8):
#             b = a + b
#         elif (norm(cross(axis,[1,1,1])) < 1e-8):
#             a = v1 + v2
#         elif (norm(cross(axis,[1,1,1])) < 1e-8):
#             if dot(a,b) < 0:
#                 b = a + b
#             b = a + b
#         if (abs(norm(a) - norm(b)) < 1e-8):
#             raise RuntimeError ('the tow vectors are identical!')

#        return a.T, b.T
        return n_list
    
    @staticmethod
    def getsigmas(uvw, limit):
        """
        ---by Haddian---
        prints a list of smallest sigmas/angles for a given axis(uvw).
        """
        sigmas = []
        thetas = []
        for i in range(limit):

            tt = get_theta_m_n_list(uvw, i)
            if len(tt) > 0 and i > 1:
                theta, m, n = tt[0]
                sigmas.append(i)
                thetas.append(degrees(theta))                
        return sigmas, thetas
    
    @staticmethod
    def get_expansion_xyz(cell,cell_size):
        cell_x, cell_y, cell_z = cell_size
        exp_x = ceil(cell_x/norm(cell[:,0]))
        exp_y = ceil(cell_y/norm(cell[:,1]))
        exp_z = ceil(cell_z/norm(cell[:,2]))
        return exp_x, exp_y, exp_z
    
    # compute_stgb → compute_bicrystal, plane_nを変える必要がある
    def get_gb_files(self,axis,plane_n,hkl,delivery_point):
        basedir = os.getcwd()
        hkl_pri = MID(self.primitive_interface.lattice_1,plane_n)
        self.primitive_interface.compute_bicrystal(hkl_pri, lim = 20, normal_ortho = True, plane_ortho = True, tol = 1e-5)
#         self.primitive_interface.compute_bicrystal(axis, plane_n, lim = 10, tol = 1e-5)
        half_cell = dot(self.primitive_interface.lattice_1, self.primitive_interface.bicrystal_U1)
        x,y,z = self.get_expansion_xyz(half_cell,self.calc_condition.cell_size)
        xhi = 2*x*norm(half_cell[:,0])


        hkl_intlist = list(map(int,hkl.split("_")))
        hkl_x, hkl_y, hkl_z = hkl_intlist
        axis_x, axis_y, axis_z = axis
        dirname = f"{axis_x}{axis_y}{axis_z}_{hkl_x}_{hkl_y}_{hkl_z}_gb"

        final_dirname = os.path.join(delivery_point,dirname)
        os.makedirs(final_dirname,exist_ok=True)
        os.chdir(final_dirname) 

        self.primitive_interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)

        CNID = dot(self.primitive_interface.orient, self.primitive_interface.CNID)
        v1 = np.array([0,1.,0])*CNID[:,0][1] + np.array([0,0,1.])*CNID[:,0][2]
        v2 = np.array([0,1.,0])*CNID[:,1][1] + np.array([0,0,1.])*CNID[:,1][2]

        n1 = int(ceil(norm(v1)/0.2))
        n2 = int(ceil(norm(v2)/0.2))
        write_trans_file(v1,v2,n1,n2)

        print(hkl_intlist)
        self.calc_condition.generate_files(axis=axis,hkl=hkl_intlist,xhi=xhi)


        os.chdir(f"{basedir}")
        
    @staticmethod    
    def order_indice(test):
        test = np.array(test)
        test = np.sort(test)
        x, y, z = test
        if x!=y:
            test = np.array([z,y,x])
        else:
            pass
        return test
    
    @staticmethod
    def v_expand(lattice):
        lim2 = 10
        x = np.arange(-lim2, lim2, 1)
        y = x
        z = x
        indice = (np.stack(np.meshgrid(x, y, z)).T).reshape(len(x) ** 3, 3)
        indice_0 = indice[np.where(np.sum(abs(indice), axis=1) != 0)[0]]
        indice_0 = indice_0[np.where(np.gcd.reduce(indice_0, axis=1) == 1)[0]]
        ltc_p = dot(indice_0, lattice.T)
        return ltc_p
    
    @staticmethod
    def get_100_angle(gbname):
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        return mis_angle
    
    @staticmethod
    def get_110_angle(gbname):
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        gbname = GBFactory.order_indice(gbname)
        x, y, z = gbname
        tang = z/np.sqrt(x**2 + y**2)
        mis_angle = 2*(np.arctan(tang)/np.pi*180)
        mis_angle = abs(mis_angle)  
        return mis_angle
    
    @staticmethod
    def get_111_angle(gbname):
        misangle_list = []
        baselist = []
        for i in list(itertools.permutations([1,-1,0])):
            baselist.append(list(i))     
        gbname = gbname.split("_")
        gbname = list(map(float, gbname))
        gbname = [-gbname[0],gbname[1],gbname[2]]
        misorientation_list = []
        for base in baselist:
            cos = dot(gbname,base)/(norm(gbname)*norm(base))
            mis_orientation = 2*(np.arccos(cos)/np.pi*180)
            mis_orientation =  abs(mis_orientation)
            misorientation_list.append(mis_orientation)
        mis_angle = min(misorientation_list)
        return mis_angle
    
    def get_gbilst(self):
        hkl_list = list(self.gb_dict.keys())
        gblist = Grainboundary_list()
        for each_hkl in hkl_list:
            if self.axis==[1,0,0]:
                tilt_angle = GBFactory.get_100_angle(each_hkl)
            elif self.axis==[1,1,0]:
                tilt_angle = GBFactory.get_110_angle(each_hkl)
            else:
                tilt_angle = GBFactory.get_111_angle(each_hkl)
            
            gblist.resister(hkl=each_hkl,tilt_angle=tilt_angle)
        
        self.gblist = gblist

In [16]:
lattice = np.array([[2,1,0],
                   [1,-2,0],
                   [0,0,1]])

In [12]:
lim = 2
x = np.arange(-lim, lim, 1)
y = x
z = x
indice = (np.stack(np.meshgrid(x, y, z)).T).reshape(len(x) ** 3, 3)
indice_0 = indice[np.where(np.sum(abs(indice), axis=1) != 0)[0]]
indice_0 = indice_0[np.where(np.gcd.reduce(indice_0, axis=1) == 1)[0]]
ltc_p = dot(indice_0, lattice.T)

In [17]:
def get_100_angle(gbname):
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    tang = min([float(gbname[1])/float(gbname[0]),float(gbname[0])/float(gbname[1])])
    mis_angle = 2*(np.arctan(tang)/np.pi*180)
    return mis_angle

In [18]:
def get_110_angle(gbname):
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    gbname = GBfactory.order_indice(gbname)
    x, y, z = gbname
    tang = z/np.sqrt(x**2 + y**2)
    mis_angle = 2*(np.arctan(tang)/np.pi*180)
    mis_angle = abs(mis_angle)  
    return mis_angle

In [19]:
def get_111_angle(gbname):
    misangle_list = []
    baselist = []
    for i in list(itertools.permutations([1,-1,0])):
        baselist.append(list(i))     
    gbname = gbname.split("_")
    gbname = list(map(float, gbname))
    gbname = [-gbname[0],gbname[1],gbname[2]]
    misorientation_list = []
    for base in baselist:
        cos = dot(gbname,base)/(norm(gbname)*norm(base))
        mis_orientation = 2*(np.arccos(cos)/np.pi*180)
        mis_orientation =  abs(mis_orientation)
        misorientation_list.append(mis_orientation)
    mis_angle = min(misorientation_list)
    return mis_angle

In [20]:
class Bcc:
    @staticmethod
    def vanishing_rule(hkl):
        print(hkl)
        x, y, z = hkl
        if ((x + y + z)%2==1):
            one_two = 2
        else:
            one_two = 1
        return one_two

In [21]:
class Crystal_structure:
    def __init__(self,vanishing_rule):
        self.vanishing_rule = vanishing_rule

In [22]:
bcc = Crystal_structure(Bcc.vanishing_rule)

In [23]:
class Fcc:
    @staticmethod
    def vanishing_rule(hkl):
        x, y, z = hkl
        if ((x%2==1) & (y%2==1) & (z%2==1)):
            one_two = 1    
        else:
            one_two = 2
        return one_two

In [24]:
fcc = Crystal_structure(Fcc.vanishing_rule)

In [40]:
element = "Ag"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Ag_meam = Element2(element,initializing_dir,fcc)

In [41]:
element = "Al"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Al_meam = Element2(element,initializing_dir,fcc)

In [24]:
element = "Au"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Au_meam = Element2(element,initializing_dir,fcc)

In [25]:
element = "Cu"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Cu_meam = Element2(element,initializing_dir,fcc)

In [26]:
element = "Ni"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Ni_meam = Element2(element,initializing_dir,fcc)

In [27]:
element = "Pb"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Pb_meam = Element2(element,initializing_dir,fcc)

In [28]:
element = "Pd"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Pd_meam = Element2(element,initializing_dir,fcc)

In [40]:
element = "Pt"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Pt_meam = Element2(element,initializing_dir,fcc)

In [30]:
element = "Fe"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Fe_meam = Element2(element,initializing_dir,bcc)

In [31]:
element = "Mo"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Mo_meam = Element2(element,initializing_dir,bcc)

In [32]:
element = "Nb"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Nb_meam = Element2(element,initializing_dir,bcc)

In [20]:
element = "Ta"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
Ta_meam = Element2(element,initializing_dir,bcc)

In [34]:
element = "V"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
V_meam = Element2(element,initializing_dir,bcc)

In [19]:
element = "W"
initializing_dir = f"/homenfs1/yhata/calc/calc_condition/meam/{element}"
W_meam = Element2(element,initializing_dir,bcc)

In [35]:
Grade = collections.namedtuple("Grade",("score","weight"))

In [36]:
class Subject(object):
    def __init__(self):
        self._grades = []
    
    def report_grade(self,score,weight):
        self._grades.append(Grade(score,weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [37]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self,name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [38]:
class Gradebook(object):
    def __init__(self):
        self._students = {}
        
    def student(self,name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [44]:
book = Gradebook()
albert = book.student("Albert Einstein")
math = albert.subject("Math")
math.report_grade(score=80,weight=0.10)

In [54]:
Grainboundary = collections.namedtuple("Grainboundary",("hkl","tilt_angle"))

In [186]:
class Grainboundary_list(object):
    def __init__(self):
        self._grainboundarylist = []
    
    # これがセッター
    def resister(self,hkl,tilt_angle):
        self._grainboundarylist.append(Grainboundary(hkl,tilt_angle))
        
    @property
    def grainboundarylist(self):
        return self._grainboundarylist
        
    @property
    def hkl(self):
        return np.array([i.hkl for i in self._grainboundarylist])
    
    @property
    def tilt_angle(self):
        return np.array([i.tilt_angle for i in self._grainboundarylist])

Calc_condition has a Element\
Calc_condition has a Protoin\
という

SamplingMethodCreater.calc_conditonはメソッドだから引数をとることができる\
その引数に元素(ポテンシャル)によって決まるtextを入れれば良い

In [26]:
fix_bulk_region = 50
middle_region = 50
start_bulk = 160
end_bulk = 165
cell_size = [100,20,20]

In [33]:
class SamplingMethodCreator_deletelayer_MEAM_shift_energytot:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self,element):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        # 緩和条件
        relaxation = f"""
# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000
        """
        

        calc_condition = f"""
variable squared_hkl1 equal {self.squared_hkl}

variable one_two1 equal {self.one_two_hkl}

variable n_layer1 equal {self.squared_hkl}

variable one_two_axis1 equal {self.one_two_axis}

variable squared_axis1 equal {self.squared_axis}

variable norm_hkl1 equal sqrt(v_squared_hkl1)

variable norm_axis1 equal sqrt(v_squared_axis1)



variable squared_hkl2 equal {self.squared_hkl}

variable one_two2 equal {self.one_two_hkl}

variable n_layer2 equal {self.squared_hkl}

variable one_two_axis2 equal {self.one_two_axis}

variable squared_axi2 equal {self.squared_axis}

variable norm_hkl2 equal sqrt(v_squared_hkl2)

variable norm_axis2 equal sqrt(v_squared_axis2)


# end of defining looping parameter------------------
# ------------- define parameter for each loop -------------

label loopa # a label where loop a starts
variable a loop ${{n_layer1}} # define a 'loop' variable
label loopb # b label where loop a starts
variable b loop ${{n_layer2}} # define b 'loop' variable
label loopc # c label where loop b starts
variable c loop 2 # define c 'loop' variable

variable dx1 equal "((v_norm_hkl1) / (v_squared_hkl1)) / (v_one_two1) * v_lattice_param"

variable dx2 equal "((v_norm_hkl2) / (v_squared_hkl2)) / (v_one_two2) * v_lattice_param"

variable tx1 equal "(v_dx1*(v_a-1))" # displacement in x direction
variable ty1 equal 0 # displacement in y direction 
variable tz1 equal 0 # displacement in z direction

variable tx2 equal "(-v_dx2*(v_b-1))" # displacement in x direction
variable ty2 equal 0 # displacement in y direction 
variable tz2 equal 0 # displacement in z direction

variable dy equal "((v_norm_axis1) / (v_squared_axis1)) / (v_one_two_axis1) / 2 * v_lattice_param"

variable delta_y equal "v_dy * (v_c-1)"




clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam {element} {element}.meam {element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # {element}

include ./blockfile2

group right_dash_dash region right
displace_atoms right_dash_dash move 0.0 ${{delta_y}} 0.0 units box

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in {element} crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

{relaxation}

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}}_${{c}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 


#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{c}} ${{gbemJm2}} ${{new_gbe}}" append results

# inner loop update firstly
next c # update the loop variable
jump proto.in loopc # jump to the label
variable c delete # after finishing the inner loop delete the loop variable of it

# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [22]:
class SamplingMethodCreator_cnid_MEAM_energytot:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl
        
    def calc_condition(self,element):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        # 緩和条件
        relaxation = f"""
# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1 zero linear
fix fixbulk1 fixbulk1 setforce 0.0 0.0 0.0

velocity fixbulk2 zero linear
fix fixbulk2 fixbulk2 setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000
        """
        
        
        calc_condition = f"""
# define looping parameter------------------ 
include ./paras 

label loopa # a label where loop a starts
variable a loop ${{na}} # define a 'loop' variable
label loopb # b label where loop b starts
variable b loop ${{nb}} # define b 'loop' variable



variable ty equal "((v_a-1) * v_cnidv1y + (v_b-1) * v_cnidv2y)" # displacement in y direction
variable tz equal "((v_a-1) * v_cnidv1z + (v_b-1) * v_cnidv2z)" # displacement in z direction
variable tx equal 0 


# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam {element} {element}.meam {element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # {element} 

# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle reduce sum c_eng
compute bulk_dis bulk displace/atom 
compute hulk_dis_ave_x bulk reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in {element} crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 

# ---------- Run Minimization ---------------------
reset_timestep 0

displace_atoms right move 0 ${{ty}} ${{tz}} units box

{relaxation} 

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final${{a}}_${{b}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
print "${{a}} ${{b}} ${{tx}} ${{ty}} ${{tz}} ${{gbemJm2}} ${{new_gbe}}" append results


# inner loop update firstly
next b # update the loop variable
jump proto.in loopb # jump to the label
variable b delete # after finishing the inner loop delete the loop variable of it

#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition

    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [32]:
class SamplingMethodCreator_deletelayer_twist_energytot:
    def __init__(self,axis,hkl):
        # この引数のself にうまいこと一仕事させたい
        self.squared_hkl = np.square(hkl).sum()
        self.squared_axis = np.square(axis).sum()
        print(self.squared_hkl)
        #self.sigma = self.get_odd_num(self.squared_hkl)
        self.axis = axis
        self.hkl = hkl

    def calc_condition(self,element):
        self.one_two_hkl = self.vanishing_rule(self.hkl)
        self.one_two_axis = self.vanishing_rule(self.axis)
        relaxation = f"""
# ---------- Run Minimization ---------------------
reset_timestep 0

velocity fixbulk1_dash zero linear
fix fixbulk1 fixbulk1_dash setforce 0.0 0.0 0.0

velocity fixbulk2_dash zero linear
fix fixbulk2 fixbulk2_dash setforce NULL NULL NULL

min_style cg 
minimize 1.0e-10 1.0e-10 50000 100000
        """
        
        
        sampling_points = int(self.squared_axis * self.one_two_axis)
        sampling_points = np.arange(sampling_points)
        sampling_points = itertools.combinations_with_replacement(sampling_points,2)
        sampling_points = np.array(list(map(list, list(sampling_points)))).T

        commands = np.array([["variable", "nr", "index"],
                            ["variable","nl","index"]])
        final_mat = np.hstack((commands,sampling_points))
        final_mat = "\n".join([" ".join(i) for i in final_mat.tolist()])
        n_layer = int(len(sampling_points.T))
        
        # 抽象基底クラスのcreatorの立ち位置にする
        calc_condition = f"""
variable one_two_axis equal {self.one_two_axis}

variable squared_axis equal {self.squared_axis}

variable norm_axis equal sqrt(v_squared_axis)

variable n_layer equal {n_layer}

{final_mat}

label loopa # a label where loop a starts
variable a loop ${{n_layer}} # define a 'loop' variable

variable dx equal "((v_norm_axis) / (v_squared_axis)) / (v_one_two_axis) * v_lattice_param"

variable tx_r equal "-(v_dx*(v_nr))" # displacement in x direction

variable tx_l equal "(v_dx*(v_nl))" # displacement in x direction

# end end of defining looping parameter------------------



clear

#Initialize Simulation --------------------- 
units metal 
dimension 3 
boundary s p p
atom_style atomic
atom_modify map array
# ---------- Create Atoms --------------------- 
read_data ./atominfile
include ./blockfile

# ---------- Define Interatomic Potential --------------------- 
pair_style meam
pair_coeff * * library.meam {element} {element}.meam {element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes

mass 1 ${{element_mass}} # {element}

displace_atoms right move ${{tx_r}} 0 0 units box
group left_dash region left 
group deleted_atoms_right subtract left_dash left
displace_atoms deleted_atoms_right move ${{startright}} 0 0 units box

displace_atoms left move ${{tx_l}} 0 0 units box
group right_dash region right
group deleted_atoms_left subtract right_dash right
displace_atoms deleted_atoms_left move -${{startright}} 0 0 units box

group fixbulk1_dash region fixbulk1 
group fixbulk2_dash region fixbulk2 
group middle_dash region middle 
group bulk_dash region bulk


# ---------- Compute properties of bulk --------------------- 
#0.excess energy
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 
compute emiddle middle_dash reduce sum c_eng
compute bulk_dis bulk_dash displace/atom 
compute hulk_dis_ave_x bulk_dash reduce ave c_bulk_dis[1]
# ---------- Calculate excess values ---------------------
#per atom properties in {element} crystal 
variable minimumenergy equal ${{minimumenergy}}
variable gbarea equal "ly * lz" 


{relaxation}

#--------- RBT right atoms -------- 

thermo 10000 
thermo_style custom step lx ly lz c_emiddle temp c_hulk_dis_ave_x c_eatoms
dump 		1 all custom 1 final_${{a}} id type x y z c_eng 
run 0

#4.excess energy
variable esum equal "v_minimumenergy * count(middle)" 
variable xseng equal "c_emiddle - (v_minimumenergy * count(middle))" 
variable gbe equal "(c_emiddle - (v_minimumenergy * count(middle)))/v_gbarea" 
variable gbemJm2 equal ${{gbe}}*16021.7733 
variable gbernd equal round(${{gbemJm2}}) 
variable ave_dis_x equal c_hulk_dis_ave_x
variable eng_tot equal "(c_eatoms - v_minimumenergy * count(all))/v_gbarea"
#variable eng_tot equal "c_eatoms"
variable e_tot equal ${{eng_tot}}
variable num_all  equal "count(all)"
variable new_gbe equal ${{e_tot}}*16021.7733

#----------- output calculation result of each loop into results file 
#print "Grain Boundary energy (meV) = ${{gbemJm2}};"
#print "All done!" 

#----------- output calculation result of each loop into results file 
print "${{a}} ${{nl}} ${{nr}} ${{gbemJm2}} ${{new_gbe}}" append results

next nl
next nr
#then go to the outter loop
next a
jump proto.in loopa
        """
        return calc_condition
    
    @classmethod    
    def get_odd_num(cls,num):
        if num%2==1:
            return num
        num = int(num/2)
        return cls.get_odd_num(num)        
    
    def vanishing_rule(self,indice):
        # これは上書きする予定
        pass

In [ ]:
element_name_list = ["Ag","Al","Au","Cu","Ni","Pb","Pd","Pt","Fe","Mo","Nb","Ta","V","W"]
element_meam_list = [Ag_meam,Al_meam,Au_meam,Cu_meam,Ni_meam,Pb_meam,Pd_meam,Pt_meam,Fe_meam,Mo_meam,Nb_meam,Ta_meam,V_meam,W_meam]


In [47]:
element_meam_list = [Ag_meam,Al_meam]

In [48]:
sampling_method = SamplingMethodCreator_deletelayer_MEAM_shift_energytot
axis = [1,0,0]
max_sigma = 10

In [60]:
sampling_method = SamplingMethodCreator_cnid_MEAM_energytot
axis = [1,0,0]
max_sigma = 10

In [66]:
sampling_method = SamplingMethodCreator_deletelayer_twist_energytot
axis = [1,0,0]
max_sigma = 10

In [ ]:

for element_meam in element_meam_list:
    calc_condition = Calc_Condition(element=element_meam,SamplingMethod=sampling_method,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)
    gbfactory = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition)
    for hkl in list(gbfactory.gb_dict.keys()):
        gbfactory.create(hkl,delivery_point=f"/homenfs1/yhata/calc/220712/{element_meam.name}")    

In [34]:
sampling_method = SamplingMethodCreator_deletelayer_MEAM_shift_energytot
axis = [1,1,0]
max_sigma = 5

In [35]:
calc_condition = Calc_Condition(element=Cu_meam,SamplingMethod=sampling_method,cell_size=cell_size,fix_bulk_region=fix_bulk_region,middle_region=middle_region,start_bulk=start_bulk,end_bulk=end_bulk)
gbfactory = GBFactory.generate_hkl_list(axis,max_sigma,calc_condition)

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.62000000e+00 3.62000000e+00 4.43322141e-16]
rotation matrix = [[ 0.66666667  0.33333333  0.66666667]
 [ 0.33333333  0.66666667 -0.66666667]
 [-0.66666667  0.66666667  0.33333333]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  1]
 [ 1 -1 -1]
 [ 0  1 -1]]; sigma_1 = 3

U2 = 
[[ 1 -1  1]
 [ 1  0 -1]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1. -0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62000000e+00 3.62000000e+00 4.43322141e-16] ; theta = 70.52877927

[20  9  1]
[20  7  3]
[20  7  3]
[20  9  1]
[20 11  1]
[20 13  3]
[20 17  7]
[20 19  9]
[19 11  0]
[19 10  1]
[19  9  2]
[19  8  3]
[19  7  4]
[19  6  5]
[19  6  5]
[19  7  4]
[19  8  3]
[19  9  2]
[19 10  1]
[19 11  0]
[19 12  1]
[19 13  2]
[19 14  3]
[19 15  4]
[19 16  5]
[19 17  6]
[19 18  7]
[19 19  8]
[18 11  1]
[6 3 1]
[18  7  5]
[18  7  5]
[6 3 1]
[18 11  1]
[18 13  1]
[6 5 1]
[18 17  5]
[19 18  

[13  6  2]
[14  6  1]
[5 2 0]
[16  6  1]
[19  5  3]
[18  5  2]
[17  5  1]
[16  5  0]
[15  5  1]
[14  5  2]
[13  5  3]
[12  5  4]
[11  5  5]
[10  6  5]
[9 7 5]
[8 8 5]
[9 7 5]
[10  6  5]
[11  5  5]
[12  5  4]
[13  5  3]
[14  5  2]
[15  5  1]
[16  5  0]
[20  4  3]
[19  4  2]
[18  4  1]
[17  4  0]
[16  4  1]
[15  4  2]
[14  4  3]
[13  4  4]
[12  5  4]
[11  6  4]
[10  7  4]
[9 8 4]
[9 8 4]
[10  7  4]
[11  6  4]
[12  5  4]
[13  4  4]
[14  4  3]
[15  4  2]
[16  4  1]
[7 1 1]
[20  3  2]
[19  3  1]
[6 1 0]
[17  3  1]
[16  3  2]
[5 1 1]
[14  4  3]
[13  5  3]
[4 2 1]
[11  7  3]
[10  8  3]
[3 3 1]
[10  8  3]
[11  7  3]
[4 2 1]
[13  5  3]
[14  4  3]
[5 1 1]
[16  3  2]
[22  3  2]
[21  2  2]
[20  2  1]
[19  2  0]
[18  2  1]
[17  2  2]
[16  3  2]
[15  4  2]
[14  5  2]
[13  6  2]
[12  7  2]
[11  8  2]
[10  9  2]
[10  9  2]
[11  8  2]
[12  7  2]
[13  6  2]
[14  5  2]
[15  4  2]
[16  3  2]
[23  3  1]
[22  2  1]
[21  1  1]
[20  1  0]
[19  1  1]
[18  2  1]
[17  3  1]
[16  4  1]
[15  5  1]
[14  6  1]
[13  

[7 4 1]
[5 4 3]
[5 4 3]
[7 4 1]
[9 4 1]
[11  4  3]
[13  5  4]
[5 2 1]
[14  5  3]
[13  4  3]
[4 1 1]
[11  3  2]
[10  3  1]
[3 1 0]
[8 3 1]
[7 3 2]
[2 1 1]
[5 4 3]
[5 4 3]
[2 1 1]
[7 3 2]
[8 3 1]
[3 1 0]
[10  3  1]
[11  3  2]
[4 1 1]
[13  4  3]
[15  5  2]
[13  3  2]
[11  2  1]
[9 2 1]
[7 3 2]
[5 5 2]
[7 3 2]
[9 2 1]
[11  2  1]
[13  3  2]
[17  6  1]
[16  5  1]
[15  4  1]
[14  3  1]
[13  2  1]
[12  1  1]
[11  1  0]
[10  1  1]
[9 2 1]
[8 3 1]
[7 4 1]
[6 5 1]
[6 5 1]
[7 4 1]
[8 3 1]
[9 2 1]
[10  1  1]
[11  1  0]
[12  1  1]
[13  2  1]
[17  5  0]
[5 1 0]
[13  1  0]
[11  1  0]
[3 1 0]
[7 5 0]
[7 5 0]
[3 1 0]
[11  1  0]
[13  1  0]
[19  6  1]
[18  5  1]
[17  4  1]
[16  3  1]
[15  2  1]
[14  1  1]
[13  1  0]
[12  1  1]
[11  2  1]
[10  3  1]
[9 4 1]
[8 5 1]
[7 6 1]
[7 6 1]
[8 5 1]
[9 4 1]
[10  3  1]
[11  2  1]
[12  1  1]
[13  1  0]
[19  5  2]
[17  3  2]
[15  2  1]
[13  2  1]
[11  3  2]
[9 5 2]
[7 7 2]
[9 5 2]
[11  3  2]
[13  2  1]
[7 2 1]
[20  5  3]
[19  4  3]
[6 1 1]
[17  3  2]
[16  3  1]
[5 1 0]


[2 1 0]
[1 1 1]
[2 1 0]
[3 1 1]
[4 2 1]
[5 3 1]
[6 4 1]
[7 5 1]
[8 6 1]
[9 7 1]
[10  8  1]
[4 3 0]
[11  8  0]
[10  7  0]
[3 2 0]
[8 5 0]
[7 4 0]
[2 1 0]
[5 2 0]
[4 1 0]
[1 0 0]
[2 1 0]
[2 1 0]
[1 0 0]
[4 1 0]
[5 2 0]
[2 1 0]
[7 4 0]
[8 5 0]
[3 2 0]
[10  7  0]
[13  9  1]
[12  8  1]
[11  7  1]
[10  6  1]
[9 5 1]
[8 4 1]
[7 3 1]
[6 2 1]
[5 1 1]
[4 1 0]
[3 1 1]
[2 2 1]
[3 1 1]
[4 1 0]
[5 1 1]
[6 2 1]
[7 3 1]
[8 4 1]
[9 5 1]
[10  6  1]
[14  9  2]
[13  8  2]
[12  7  2]
[11  6  2]
[10  5  2]
[9 4 2]
[8 3 2]
[7 2 2]
[6 2 1]
[5 2 0]
[4 2 1]
[3 2 2]
[3 2 2]
[4 2 1]
[5 2 0]
[6 2 1]
[7 2 2]
[8 3 2]
[9 4 2]
[10  5  2]
[5 3 1]
[14  8  3]
[13  7  3]
[4 2 1]
[11  5  3]
[10  4  3]
[3 1 1]
[8 3 2]
[7 3 1]
[2 1 0]
[5 3 1]
[4 3 2]
[1 1 1]
[4 3 2]
[5 3 1]
[2 1 0]
[7 3 1]
[8 3 2]
[3 1 1]
[10  4  3]
[16  9  4]
[15  8  4]
[14  7  4]
[13  6  4]
[12  5  4]
[11  4  4]
[10  4  3]
[9 4 2]
[8 4 1]
[7 4 0]
[6 4 1]
[5 4 2]
[4 4 3]
[4 4 3]
[5 4 2]
[6 4 1]
[7 4 0]
[8 4 1]
[9 4 2]
[10  4  3]
[17  9  5]
[16  8  5]
[15  7

[13 11  8]
[11  9  8]
[9 8 7]
[8 7 5]
[8 5 3]
[8 3 1]
[8 1 1]
[8 3 1]
[8 5 3]
[8 7 5]
[5 4 3]
[14 11  9]
[13 10  9]
[4 3 3]
[11  9  8]
[10  9  7]
[3 3 2]
[9 8 5]
[9 7 4]
[3 2 1]
[9 5 2]
[9 4 1]
[3 1 0]
[9 2 1]
[9 2 1]
[3 1 0]
[9 4 1]
[9 5 2]
[3 2 1]
[9 7 4]
[15 11 10]
[13 10  9]
[11 10  7]
[10  9  5]
[10  7  3]
[10  5  1]
[10  3  1]
[10  3  1]
[10  5  1]
[10  7  3]
[17 12 11]
[16 11 11]
[15 11 10]
[14 11  9]
[13 11  8]
[12 11  7]
[11 11  6]
[11 10  5]
[11  9  4]
[11  8  3]
[11  7  2]
[11  6  1]
[11  5  0]
[11  4  1]
[11  3  2]
[11  3  2]
[11  4  1]
[11  5  0]
[11  6  1]
[11  7  2]
[13  7  3]
[12  7  4]
[11  7  5]
[10  7  6]
[9 7 7]
[8 8 7]
[9 7 7]
[10  7  6]
[11  7  5]
[12  7  4]
[13  7  3]
[14  7  2]
[15  7  1]
[16  7  0]
[17  7  1]
[18  7  2]
[19  7  3]
[20  7  4]
[21  7  5]
[22  7  6]
[13  6  2]
[11  6  4]
[10  6  5]
[8 7 6]
[8 7 6]
[10  6  5]
[11  6  4]
[13  6  2]
[14  6  1]
[16  6  1]
[17  6  2]
[19  6  4]
[20  6  5]
[13  5  1]
[12  5  2]
[11  5  3]
[10  5  4]
[9 5 5]
[8 6 5]
[7 7

[7 6 2]
[7 7 1]
[8 7 0]
[9 7 1]
[10  7  2]
[11  7  3]
[12  7  4]
[15  8  8]
[14  8  7]
[13  8  6]
[12  8  5]
[11  8  4]
[10  8  3]
[9 8 2]
[8 8 1]
[8 7 0]
[8 6 1]
[8 5 2]
[8 4 3]
[8 4 3]
[8 5 2]
[8 6 1]
[8 7 0]
[8 8 1]
[9 8 2]
[10  8  3]
[11  8  4]
[5 3 3]
[14  9  8]
[13  9  7]
[4 3 2]
[11  9  5]
[10  9  4]
[3 3 1]
[9 8 2]
[9 7 1]
[3 2 0]
[9 5 1]
[9 4 2]
[3 1 1]
[9 4 2]
[9 5 1]
[3 2 0]
[9 7 1]
[9 8 2]
[3 3 1]
[10  9  4]
[14 10  9]
[13 10  8]
[12 10  7]
[11 10  6]
[10  9  4]
[10  8  3]
[10  7  2]
[10  6  1]
[10  4  1]
[10  3  2]
[10  3  2]
[10  4  1]
[10  6  1]
[10  7  2]
[10  8  3]
[10  9  4]
[15 11 11]
[14 11 10]
[13 11  9]
[12 11  8]
[11 11  7]
[11 10  6]
[11  9  5]
[11  8  4]
[11  7  3]
[11  6  2]
[11  5  1]
[11  4  0]
[11  3  1]
[11  2  2]
[11  3  1]
[11  4  0]
[11  5  1]
[11  6  2]
[11  7  3]
[11  8  4]
[5 4 4]
[14 12 11]
[13 12 10]
[4 4 3]
[12 11  8]
[12 10  7]
[4 3 2]
[12  8  5]
[12  7  4]
[4 2 1]
[12  5  2]
[12  4  1]
[4 1 0]
[12  2  1]
[12  2  1]
[4 1 0]
[12  4  1]
[12  5  2]


[13  8  3]
[15  8  1]
[17  8  1]
[6 3 1]
[17  9  2]
[16  9  1]
[5 3 0]
[14  9  1]
[13  9  2]
[4 3 1]
[11  9  4]
[10  9  5]
[3 3 2]
[9 8 7]
[9 8 7]
[3 3 2]
[10  9  5]
[11  9  4]
[4 3 1]
[13  9  2]
[14  9  1]
[5 3 0]
[16  9  1]
[17 10  3]
[15 10  1]
[13 10  1]
[11 10  3]
[10  9  5]
[10  7  7]
[10  9  5]
[11 10  3]
[13 10  1]
[15 10  1]
[18 11  5]
[17 11  4]
[16 11  3]
[15 11  2]
[14 11  1]
[13 11  0]
[12 11  1]
[11 11  2]
[11 10  3]
[11  9  4]
[11  8  5]
[11  7  6]
[11  7  6]
[11  8  5]
[11  9  4]
[11 10  3]
[11 11  2]
[12 11  1]
[13 11  0]
[14 11  1]
[17 12  5]
[5 4 1]
[13 12  1]
[12 11  1]
[4 3 1]
[12  7  5]
[12  7  5]
[4 3 1]
[12 11  1]
[13 12  1]
[18 13  7]
[17 13  6]
[16 13  5]
[15 13  4]
[14 13  3]
[13 13  2]
[13 12  1]
[13 11  0]
[13 10  1]
[13  9  2]
[13  8  3]
[13  7  4]
[13  6  5]
[13  6  5]
[13  7  4]
[13  8  3]
[13  9  2]
[13 10  1]
[13 11  0]
[13 12  1]
[17 14  7]
[15 14  5]
[14 13  3]
[14 11  1]
[14  9  1]
[14  7  3]
[14  5  5]
[14  7  3]
[14  9  1]
[14 11  1]
[6 5 3]
[17 1

In [36]:
gb_asym_list = ["7_7_4","5_5_2","1_1_0","2_2_1","5_5_4"]

In [37]:
for hkl in gb_asym_list:
    gbfactory.create(hkl,delivery_point=f"/homenfs1/yhata/calc/221110/Cu_aysm_ct")

[-2  0  0]
70.52877936550931
[[-1.81000000e+00 -1.81000000e+00  1.10830535e-16]
 [-1.81000000e+00  0.00000000e+00 -1.81000000e+00]
 [ 0.00000000e+00 -1.81000000e+00 -1.81000000e+00]]
[3.62 3.62 0.  ]
rotation matrix = [[ 0.66666667  0.33333333  0.66666667]
 [ 0.33333333  0.66666667 -0.66666667]
 [-0.66666667  0.66666667  0.33333333]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0 -1]
 [ 0  1 -1]
 [ 0  0  3]]; sigma_1 = 3

U2 = 
[[ 1  0  1]
 [ 0  0 -3]
 [ 0  1  1]]; sigma_2 = 3

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 0.  ] ; theta = 70.52877927

cell 1:
[[-2  1 -7]
 [-5  0 11]
 [ 9  0  3]]
cell 2:
[[-2  1 -7]
 [-5  0 11]
 [ 9  0  3]]
[[-2  1 -7]
 [-5  0 11]
 [ 9  0  3]]
[[ 4  1 -5]
 [-9  0 -3]
 [ 1  0 13]]
num of atoms1 = 114
[[ 1.15953070e+02  1.24877053e-16  4.23039870e-16]
 [ 2.58672959e-15  2.04778124e+01 -1.25935645e-15]
 [-5.27735383e-15  1.49503849e-16  2.73304006e+01]]
[7, 7, 4]
114
[-2  0  0]
70.52877936550931
[[-1.81000000e+00 -1.81000000e+00  1

In [89]:
os.getcwd()

'/homenfs1/yhata/calc/221110/gbfactory'